In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import numpy as np
import random
import os
from tqdm.notebook import tqdm
from sklearn.metrics import root_mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:.5f}'.format
pd.options.display.max_rows = None

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
SEED = 1044
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
DATA_PATH = '/kaggle/input/playground-series-s3e20'

train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
samplesubmission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

train.head(10)

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.shape , test.shape

In [ ]:
train['location'] = [str(x) + '_' + str(y) for x, y in zip(train.latitude, train.longitude)]

In [ ]:
train.tail(10)

In [ ]:
exampleLoc = train[train.location == '-3.299_30.301']

rollmean = exampleLoc['SulphurDioxide_SO2_column_number_density_amf'].rolling(window=2).mean()

In [ ]:
plt.figure(figsize=(15, 7))
rollmean.plot()
plt.title('Rolling mean with a window of 2 weeks for SulphurDioxide_SO2_column_number_density_amf', y=1.02, fontsize=15)
plt.xlabel('week', y=1.05, fontsize=13)
plt.ylabel('SulphurDioxide_SO2_column_number_density_amf', x=1.05, fontsize=13)
plt.show()

In [ ]:
train_roll_mean = train.sort_values(by = ['location', 'year', 'week_no']).groupby(['location'])[train.columns[5:].tolist()].rolling(window = 2).mean().reset_index()
train_roll_mean.drop(['level_1', 'emission', 'location'], axis = 1, inplace = True)
train_roll_mean.columns = [col + '_roll_mean' for col in train_roll_mean.columns]


test.latitude, test.longitude = round(test.latitude, 2), round(test.longitude, 2)
test['location'] = [str(x) + '_' + str(y) for x, y in zip(test.latitude, test.longitude)]
test_roll_mean = test.sort_values(by = ['location', 'year', 'week_no']).groupby(['location'])[test.columns[5:].tolist()].rolling(window = 2).mean().reset_index()
test_roll_mean.drop(['level_1', 'location'], axis = 1, inplace = True)
test_roll_mean.columns =  [col + '_roll_mean' for col in test_roll_mean.columns]
test_roll_mean.head()


In [ ]:
train_eng = train.sort_values(by = ['location', 'year', 'week_no'], ignore_index = True).merge(train_roll_mean, how = 'left',
                                                                                               left_index=True, right_index=True)
test_eng = test.sort_values(by = ['location', 'year', 'week_no'], ignore_index = True).merge(test_roll_mean, how = 'left',
                                                                                               left_index=True, right_index=True)
test_eng.head()

In [ ]:
X = train_eng.drop(['ID_LAT_LON_YEAR_WEEK', 'location', 'emission'], axis=1)
X = X.fillna(X.median())
y = train_eng.emission

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

xgb_reg = xgb.XGBRegressor(random_state=SEED, n_jobs=-1)

xgb_reg.fit(X_train, y_train)

y_pred = xgb_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE Score: {rmse}')

In [ ]:
X_test.head(10)

In [ ]:
X_test.info()

In [ ]:
X_test.shape

In [ ]:
prederr = X_test.copy()
prederr['emission'] = y_test
prederr['prediction'] = y_pred
prederr['error'] = abs(prederr.prediction - prederr.emission)
prederr = prederr[['latitude',	'longitude',	'year',	'week_no', 'emission', 'prediction', 'error']]
prederr.sort_values(by = 'error', ascending = False, inplace = True)
prederr.head()

In [ ]:
prederr.head(10)


In [ ]:
impo_df = pd.DataFrame({'feature': X.columns, 'importance': xgb_reg.feature_importances_}).set_index('feature').sort_values(by = 'importance', ascending = False)
impo_df = impo_df[:21].sort_values(by = 'importance', ascending = True)
impo_df.plot(kind = 'barh', figsize = (10, 10))
plt.legend(loc = 'center right')
plt.title('Feature importance', fontsize = 14)
plt.xlabel('Features', fontsize = 12)
plt.show()

In [ ]:
test_df = test_eng.drop(['ID_LAT_LON_YEAR_WEEK', 'location'], axis = 1)
test_df = test_df.fillna(test_df.median())
predictions = xgb_reg.predict(test_df)

subFile = pd.DataFrame({'ID_LAT_LON_YEAR_WEEK': test_eng.ID_LAT_LON_YEAR_WEEK, 'emission': predictions})


In [ ]:
subFile.head(20)

In [ ]:
subFile.describe()

In [ ]:
subFile['emission'] = subFile['emission'].abs()
subFile.head(10)

In [ ]:
subFile.to_csv('BaselineSubmission.csv', index = False)